<a href="https://colab.research.google.com/github/amolaka/DS-3001---Voting-Project/blob/main/Models_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
# Import libraries and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [170]:
# Import data
data = pd.read_csv('/content/voting_data_with_population-2.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data.head()

,year,state,county_fips,county_name,democrat_votes,republican_votes,total_votes,population
0,2000,VIRGINIA,51001,ACCOMACK,5092,6352,11444,38305.0
1,2000,VIRGINIA,51003,ALBEMARLE,16255,18291,34546,79236.0
2,2000,VIRGINIA,51005,ALLEGHANY,2214,2808,5022,12926.0
3,2000,VIRGINIA,51007,AMELIA,1754,2947,4701,11400.0
4,2000,VIRGINIA,51009,AMHERST,4812,6660,11472,31894.0


In [171]:
data = data.dropna()

In [172]:
train_data = data[data['year'] != 2020]
test_data = data[data['year'] == 2020]

county_names = data[data['year'] == 2020]
county_names = county_names.drop(columns = ['democrat_votes', 'republican_votes', 'year'])

train_data = train_data.drop(columns = ['state', 'county_name'])
test_data = test_data.drop(columns = ['state', 'county_name'])

# Split data into train/test:
X_train = train_data.drop(columns = ['democrat_votes', 'republican_votes'])
y_train = train_data.loc[:, ['democrat_votes', 'republican_votes']]
X_test = test_data.drop(columns = ['democrat_votes', 'republican_votes'])
y_test = test_data.loc[:, ['democrat_votes', 'republican_votes']]

In [173]:
train_data.head()

,year,county_fips,democrat_votes,republican_votes,total_votes,population
0,2000,51001,5092,6352,11444,38305.0
1,2000,51003,16255,18291,34546,79236.0
2,2000,51005,2214,2808,5022,12926.0
3,2000,51007,1754,2947,4701,11400.0
4,2000,51009,4812,6660,11472,31894.0


In [174]:
test_data.head()

,year,county_fips,democrat_votes,republican_votes,total_votes,population
670,2020,51001,7578,9172,16750,32673.0
671,2020,51003,42466,20804,63270,107405.0
672,2020,51005,2243,5859,8102,15157.0
673,2020,51007,2411,5390,7801,12953.0
674,2020,51009,5672,11041,16713,31775.0


Random Forest

In [175]:
from sklearn.ensemble import RandomForestRegressor

# Fit model:
model = RandomForestRegressor() # Build a random forest model
model.fit(X_train,y_train) # Fit the model

RandomForestRegressor()

In [176]:
# Look at R-squared values
train_data_score = model.score(X_train, y_train)
print(f'R^2 - Train: {train_data_score:.4f}')

test_data_score = model.score(X_test, y_test)
print(f'R^2 - Test: {test_data_score:.4f}')

R^2 - Train: 0.9959
R^2 - Test: 0.9457


In [177]:
# Look at predicted voting data
prediction = model.predict(X_test) # Model predictions

# Make a new datafarme
voting_predictions = pd.DataFrame()
voting_predictions['county_name'] = county_names['county_name']
voting_predictions['county_fips'] = county_names['county_fips']
voting_predictions['democrat_votes'] = prediction[:, 0]
voting_predictions['republican_votes'] = prediction[:, 1]

# Look at new dataframe
voting_predictions

,county_name,county_fips,democrat_votes,republican_votes
670,ACCOMACK,51001,6294.24,10433.81
671,ALBEMARLE,51003,34887.38,26885.95
672,ALLEGHANY,51005,3368.36,4659.93
673,AMELIA,51007,2890.90,4826.27
674,AMHERST,51009,6202.84,10316.57
...,...,...,...,...
798,SUFFOLK CITY,51800,24541.17,21108.89
799,VIRGINIA BEACH CITY,51810,95656.54,97633.19
800,WAYNESBORO CITY,51820,5427.72,4921.56
801,WILLIAMSBURG CITY,51830,4170.24,2788.64


In [178]:
# Look at total democrat vs total republican votes
print("Total Democrat Votes: " + str(voting_predictions['democrat_votes'].sum()))
print("Total Republican Votes: " + str(voting_predictions['republican_votes'].sum()))

Total Democrat Votes: 2127362.01
Total Republican Votes: 2015866.9300000002


In [179]:
voting_predictions.to_csv('voting_predictions.csv')

Random Forest with Variables

In [180]:
# Import data
data2 = pd.read_csv('/content/full_combined_data-3.csv')
data2 = data2.drop(['Unnamed: 0'], axis=1)
data2

,year,state,county_fips,county_name,democrat_votes,republican_votes,total_votes,population,White alone,Black or African American alone,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,2000,VIRGINIA,51001,ACCOMACK,5092,6352,11444,38305.0,394.0,84.0,...,6.086775,6.814543,6.274762,5.411646,5.786897,5.762051,6.833032,6.419995,5.537334,5.710427
1,2000,VIRGINIA,51003,ALBEMARLE,16255,18291,34546,79236.0,509.0,328.0,...,6.523562,6.971669,6.626718,6.342121,6.838405,6.313548,6.967909,6.938284,6.395262,6.837333
2,2000,VIRGINIA,51005,ALLEGHANY,2214,2808,5022,12926.0,75.0,113.0,...,5.579730,6.148468,5.846439,4.890349,4.779123,5.293305,6.253829,5.942799,5.036953,5.627621
3,2000,VIRGINIA,51007,AMELIA,1754,2947,4701,11400.0,152.0,212.0,...,5.796058,6.326149,5.877736,4.976734,5.187386,4.997212,6.364751,6.049733,4.927254,5.863631
4,2000,VIRGINIA,51009,AMHERST,4812,6660,11472,31894.0,323.0,162.0,...,6.016157,6.699500,6.304449,5.117994,5.370638,6.284134,6.689599,6.298949,5.365976,5.583496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,2020,VIRGINIA,51800,SUFFOLK CITY,28676,20082,48758,90093.0,320.0,721.0,...,6.102559,NaN,7.006695,6.091310,6.381816,6.198479,NaN,7.224025,6.349139,6.504288
799,2020,VIRGINIA,51810,VIRGINIA BEACH CITY,117393,105087,222480,450201.0,1288.0,1323.0,...,6.549651,NaN,8.020928,7.108244,7.296413,6.770789,NaN,7.998671,7.244942,7.540090
800,2020,VIRGINIA,51820,WAYNESBORO CITY,4961,5507,10468,22140.0,251.0,328.0,...,5.872118,NaN,6.338594,5.652489,5.894403,6.025866,NaN,6.572283,5.509388,6.059123
801,2020,VIRGINIA,51830,WILLIAMSBURG CITY,4790,1963,6753,14927.0,236.0,162.0,...,5.081404,NaN,5.587249,5.262690,5.613128,4.859812,NaN,6.182085,5.049856,5.736572


In [186]:
data2 = data2.dropna()
data2

,year,state,county_fips,county_name,democrat_votes,republican_votes,total_votes,population,White alone,Black or African American alone,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,2000,VIRGINIA,51001,ACCOMACK,5092,6352,11444,38305.0,394.0,84.0,...,6.086775,6.814543,6.274762,5.411646,5.786897,5.762051,6.833032,6.419995,5.537334,5.710427
1,2000,VIRGINIA,51003,ALBEMARLE,16255,18291,34546,79236.0,509.0,328.0,...,6.523562,6.971669,6.626718,6.342121,6.838405,6.313548,6.967909,6.938284,6.395262,6.837333
2,2000,VIRGINIA,51005,ALLEGHANY,2214,2808,5022,12926.0,75.0,113.0,...,5.579730,6.148468,5.846439,4.890349,4.779123,5.293305,6.253829,5.942799,5.036953,5.627621
3,2000,VIRGINIA,51007,AMELIA,1754,2947,4701,11400.0,152.0,212.0,...,5.796058,6.326149,5.877736,4.976734,5.187386,4.997212,6.364751,6.049733,4.927254,5.863631
4,2000,VIRGINIA,51009,AMHERST,4812,6660,11472,31894.0,323.0,162.0,...,6.016157,6.699500,6.304449,5.117994,5.370638,6.284134,6.689599,6.298949,5.365976,5.583496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,2004,VIRGINIA,51515,BEDFORD,1042,1472,2514,6299.0,176.0,201.0,...,6.047372,7.080868,6.752270,5.958425,6.093570,5.926926,7.004882,6.807935,5.846439,6.028279
239,2004,VIRGINIA,51600,FAIRFAX,5395,5045,10440,21498.0,3057.0,1025.0,...,7.678326,8.178639,7.990915,7.467942,8.152774,7.583756,8.137103,8.072467,7.604396,8.029433
241,2004,VIRGINIA,51620,FRANKLIN,1910,1613,3523,8346.0,148.0,168.0,...,6.309918,7.079184,6.720220,5.598422,5.840642,6.013715,6.896694,6.720220,5.749393,5.932245
259,2004,VIRGINIA,51760,RICHMOND,52167,21637,73804,197790.0,99.0,104.0,...,5.393628,6.733402,5.697093,4.499810,4.624973,5.087596,5.908083,5.686975,4.553877,5.204007


In [187]:
data2['year'].unique()

array([2000, 2004])

In [188]:
train_data = data2[data2['year'] == 2000]
test_data = data2[data2['year'] == 2004]

county_names = data2[data2['year'] == 2004][['county_name']]

train_data = train_data.drop(columns = ['state', 'county_name'])
test_data = test_data.drop(columns = ['state', 'county_name'])

# Split data into train/test:
X_train = train_data.drop(columns = ['democrat_votes', 'republican_votes'])
y_train = train_data.loc[:, ['democrat_votes', 'republican_votes']]
X_test = test_data.drop(columns = ['democrat_votes', 'republican_votes'])
y_test = test_data.loc[:, ['democrat_votes', 'republican_votes']]

In [189]:
train_data.head()

,year,county_fips,democrat_votes,republican_votes,total_votes,population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,2000,51001,5092,6352,11444,38305.0,394.0,84.0,32.0,44.0,...,6.086775,6.814543,6.274762,5.411646,5.786897,5.762051,6.833032,6.419995,5.537334,5.710427
1,2000,51003,16255,18291,34546,79236.0,509.0,328.0,140.0,223.0,...,6.523562,6.971669,6.626718,6.342121,6.838405,6.313548,6.967909,6.938284,6.395262,6.837333
2,2000,51005,2214,2808,5022,12926.0,75.0,113.0,5.0,70.0,...,5.579730,6.148468,5.846439,4.890349,4.779123,5.293305,6.253829,5.942799,5.036953,5.627621
3,2000,51007,1754,2947,4701,11400.0,152.0,212.0,38.0,53.0,...,5.796058,6.326149,5.877736,4.976734,5.187386,4.997212,6.364751,6.049733,4.927254,5.863631
4,2000,51009,4812,6660,11472,31894.0,323.0,162.0,85.0,51.0,...,6.016157,6.699500,6.304449,5.117994,5.370638,6.284134,6.689599,6.298949,5.365976,5.583496


In [190]:
test_data.head()

,year,county_fips,democrat_votes,republican_votes,total_votes,population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
134,2004,51001,5518,7726,13244,38305.0,394.0,84.0,32.0,44.0,...,6.086775,6.814543,6.274762,5.411646,5.786897,5.762051,6.833032,6.419995,5.537334,5.710427
135,2004,51003,22088,21189,43277,79236.0,509.0,328.0,140.0,223.0,...,6.523562,6.971669,6.626718,6.342121,6.838405,6.313548,6.967909,6.938284,6.395262,6.837333
136,2004,51005,3203,3962,7165,12926.0,75.0,113.0,5.0,70.0,...,5.579730,6.148468,5.846439,4.890349,4.779123,5.293305,6.253829,5.942799,5.036953,5.627621
137,2004,51007,1862,3499,5361,11400.0,152.0,212.0,38.0,53.0,...,5.796058,6.326149,5.877736,4.976734,5.187386,4.997212,6.364751,6.049733,4.927254,5.863631
138,2004,51009,4866,7758,12624,31894.0,323.0,162.0,85.0,51.0,...,6.016157,6.699500,6.304449,5.117994,5.370638,6.284134,6.689599,6.298949,5.365976,5.583496


In [191]:
from sklearn.ensemble import RandomForestRegressor

# Fit model:
model = RandomForestRegressor() # Build a random forest model
model.fit(X_train,y_train) # Fit the model

RandomForestRegressor()

In [192]:
# Look at R-squared values
train_data_score = model.score(X_train, y_train)
print(f'R^2 - Train: {train_data_score:.4f}')

test_data_score = model.score(X_test, y_test)
print(f'R^2 - Test: {test_data_score:.4f}')

R^2 - Train: 0.8973
R^2 - Test: 0.8393


In [194]:
# Look at predicted voting data
prediction = model.predict(X_test) # Model predictions

# Make a new datafarme
voting_predictions = pd.DataFrame()
voting_predictions['county_name'] = county_names['county_name']
voting_predictions['county_fips'] = data2['county_fips']
voting_predictions['democrat_votes'] = prediction[:, 0]
voting_predictions['republican_votes'] = prediction[:, 1]

# Look at new dataframe
voting_predictions

,county_name,county_fips,democrat_votes,republican_votes
134,ACCOMACK,51001,5109.56,8086.85
135,ALBEMARLE,51003,14758.51,21096.40
136,ALLEGHANY,51005,2800.90,3803.44
137,AMELIA,51007,2067.58,3320.76
138,AMHERST,51009,5218.00,7145.58
...,...,...,...,...
230,BEDFORD,51515,1152.30,1474.53
239,FAIRFAX,51600,42176.37,44261.74
241,FRANKLIN,51620,1629.98,1638.12
259,RICHMOND,51760,38164.42,24772.68


In [195]:
# Look at total democrat vs total republican votes
print("Total Democrat Votes: " + str(voting_predictions['democrat_votes'].sum()))
print("Total Republican Votes: " + str(voting_predictions['republican_votes'].sum()))

Total Democrat Votes: 871973.1499999999
Total Republican Votes: 1149816.31
